In [1]:
#%%
import numpy as np
import pandas as pd
from datetime import datetime,date
import time

import cx_Oracle
import csv
import math
import pandas as pd
import numpy as np

#加减年月
import dateutil.relativedelta
# from datetime import datetime
# from datetime import date

import matplotlib.pyplot as plt
import os

from datetime import *
dns_tns=cx_Oracle.makedsn('172.16.50.233',1521,service_name='dfcfstb')
conn = cx_Oracle.Connection(user='cjchenjm5',password='Jm0228_Cjsc!',dsn=dns_tns,encoding='UTF-16')

In [2]:
# 取每月最后一个交易日
# 月度调仓
def get_last_tradate(date0,date1):
    date0 = date0.strftime('%Y%m%d')
    date1 = date1.strftime('%Y%m%d')   
    sql = f'''
                select 交易日期,month
                from
                (
                    select 交易日期,month,row_number() over(partition by month order by 交易日期 desc) as rank
                    from
                    (
                        SELECT F1_1010 交易日期,substr(F1_1010,1,6) month
                        FROM wind.TB_OBJECT_1010
                        where F1_1010 <= '{date1}'
                        and F1_1010 >= '{date0}'
                    )
                )t
                where t.rank = 1
            '''
    data = pd.read_sql(sql, conn)
    return data

time_list = get_last_tradate(datetime(2010,12,31),date.today())
time_list['交易日期'] = pd.to_datetime(time_list['交易日期'])
space_month=time_list

e:\anocanda\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
def l_v_d(date0):
        # 日均成交额
    d0 = date0 - pd.DateOffset(years=1)
    date1 = d0.strftime('%Y%m%d')
    date0 = date0.strftime('%Y%m%d')
    sql=f'''
            select 
                    代码, 日均成交量, 交易所后缀
            from(
                select
                        F16_1090 代码,
                        avg(F11_1120/100000) 日均成交量,   
                        F7_0002 交易所后缀
                    from
                            wind.TB_OBJECT_1120
                    left join wind.TB_OBJECT_1090 on F2_1090 = F1_1120
                    left join wind.TB_OBJECT_0002 on F27_1090 = F4_0002
                    where
                            F2_1120 <= '{date0}' and F2_1120 >= '{date1}'
                            AND F4_1090 = 'A'  and F17_1090 <= '{date0}'
                    group by F16_1090, F7_0002
            )
                    
    '''
    data_turnover=pd.read_sql(sql,conn)
    
    # 剔除后20%
    data_turnover['日期']=date0
    data_turnover['turnover_day']=data_turnover.groupby('日期').rank(pct=True)['日均成交量']
    data_turnover=data_turnover.loc[data_turnover['turnover_day']>0.2]
    
    date0=pd.to_datetime(date0)
    d0=date0 - pd.DateOffset(months=11)
    date0=d0.strftime('%Y')+str('1231')
    date0=pd.to_datetime(date0)
    d0=date0 - pd.DateOffset(years=2)
    date1=d0.strftime('%Y%m%d')
    date0=date0.strftime('%Y%m%d')

    # 扣非ROE
    sql=f'''
        select
                代码, 日期, 净资产收益率_扣非, 交易所后缀
        from 
            (
            select 
                    F16_1090 代码,
                    F3_5034 日期,
                    F53_5034 净资产收益率_扣非,
                    F7_0002 交易所后缀, 
                    F17_1090 上市时间,
                    
                    row_number() over(partition by F16_1090, F3_5034 order by F3_5034 desc) as rank
            from
                    wind.TB_OBJECT_5034
            left join wind.TB_OBJECT_1090 on Ob_revisions_1090 = F1_5034
            left join wind.TB_OBJECT_0002 on F27_1090 = F4_0002
            
            where
                    F3_5034 <= '{date0}' and F3_5034 >= '{date1}' and F17_1090 <= '{date0}'
                    AND F4_1090 = 'A' and F3_5034 like '%1231'
            )
    '''
    data=pd.read_sql(sql,conn)

    # 扣非roe均值
    data1=data.copy()
    data2=data1.groupby('代码').mean()['净资产收益率_扣非']

    data3=data1[data1['日期']==date0].drop('净资产收益率_扣非',axis=1)
    data4=pd.DataFrame(data2)
    data4=data4.reset_index()
    data4=data4.rename(columns={'净资产收益率_扣非':'扣非roe均值'})
    data5=pd.merge(data3,data4,on='代码',how='outer') # 扣非ROE均值
    data5=data5.sort_values('代码')

    # 扣非roe标准差
    data1s=data.copy()
    data2s=data1s.groupby('代码').std()['净资产收益率_扣非']
    data3s=data1s[data1s['日期']==date0].drop('净资产收益率_扣非',axis=1)
    data4s=pd.DataFrame(data2s)
    data4s=data4s.reset_index()
    data4s=data4s.rename(columns={'净资产收益率_扣非':'扣非roe标准差'})
    data5s=pd.merge(data3s,data4s,on='代码',how='outer') # 扣非ROE标准差
    data5s=data5s.sort_values('代码')
    # 均值-标准差
    data5s['mean_std']=data5['扣非roe均值']-data5s['扣非roe标准差']
    data_roe=data5s.copy()

    # 连接成交量和roe
    data_roe=pd.merge(data_turnover,data_roe,on='代码',how='left')

    # 剔除后30%
    data_roe=data_roe.drop(data_roe[np.isnan(data_roe['mean_std'])].index) # 去除na值
    data_roe['roe_rank']=data_roe.groupby('日期_x').rank(pct=True)['mean_std']
    data_roe=data_roe.loc[data_roe['roe_rank']>0.3]

    # 分红
    sql=f'''
            select 
                代码, 分红年度,  派息比例分子,交易所后缀, rank
            from 
                (
                select 
                        F16_1090 代码,
                        F24_1093 分红年度,
                        F9_1093 派息比例分子,
                        F7_0002 交易所后缀,
                        F26_1093 除权除息日,
                        row_number() over(partition by F16_1090 order by F24_1093 desc) as rank
                        
                from
                    wind.TB_OBJECT_1093
                left join wind.TB_OBJECT_1090 on F2_1090 = F1_1093
                left join wind.TB_OBJECT_0002 on F27_1090 = F4_0002
                where 
                    F24_1093 <= '{date0}' 
                    and F24_1093 >= '{date1}' and F4_1090 = 'A' and F24_1093 like '%1231'
                )

    '''
    data_divid=pd.read_sql(sql,conn)

    data_divid1=data_divid.copy()
    data_divid2=data_divid1.drop(data_divid1[np.isnan(data_divid1['派息比例分子'])].index) # 去除没有分红的年份
    data_divid3=data_divid2.groupby('代码').filter(lambda x: len(x)==3) # 连续三年分红的股票
    data_divid4=data_divid3[data_divid3['分红年度']==date0]
    data_dividend=data_divid4.rename(columns={'分红年度':'日期'}) # 连续三年分红

    # 组合dividend和roe
    data_merge=pd.merge(data_dividend,data_roe,on='代码')

    return data_merge


In [7]:
# 选股策略_所有调仓日
pool = pd.DataFrame()
for i in range(len(space_month)):
    date = space_month['交易日期'][i]#datetime
    temp = l_v_d(date)
    pool=pool.append(temp)

e:\anocanda\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
e:\anocanda\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
e:\anocanda\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1620\3315866933.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 